In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

In [ ]:
# Get the OpenAI API key from the .env file
load_dotenv('.env', override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key = openai_api_key)

In [1]:
def get_llm_response(prompt):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "You are an AI assistant.",
            },
            {"role": "user", "content": prompt},
        ],
        temperature=0.0,
    )
    response = completion.choices[0].message.content
    return response

In [ ]:
pip install pypdf

In [ ]:
pip install pymupdf


In [ ]:
%pip install pytesseract pillow


In [ ]:
!pip install pdfplumber openai python-dotenv ipywidgets

In [ ]:
import pdfplumber
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# Step 2: File upload widget for Jupyter
upload_widget = widgets.FileUpload(accept='.pdf', multiple=False)
display(upload_widget)

In [ ]:
# Step 3: Save uploaded PDF file locally
def save_uploaded_file(upload_widget):
    if upload_widget.value and len(upload_widget.value) > 0:
        file_info = upload_widget.value[0]
        file_name = file_info['name']
        content = file_info['content']
        with open(file_name, "wb") as f:
            f.write(content)
        return file_name
    return None

In [ ]:
# After uploading, run this cell manually
file_path = save_uploaded_file(upload_widget)
print("Saved file:", file_path)

In [ ]:
# Step 4: Extract raw text from PDF using pdfplumber
def extract_text_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

In [ ]:
# Step 5: Extract form fields (includes radio buttons / checkboxes) using PyPDF2
def extract_pdf_form_fields(file_path):
    reader = PdfReader(file_path)
    fields = reader.get_fields()
    if fields:
        result = {}
        for key, val in fields.items():
            # val can be a dictionary or direct value
            if isinstance(val, dict):
                value = val.get('/V') or val.get('/AS') or None
                # Some values are NameObjects, convert to str ignoring '/'
                if value and hasattr(value, 'original_bytes'):
                    value = value.original_bytes.decode('utf-8')
                if isinstance(value, str) and value.startswith('/'):
                    value = value[1:]
                result[key] = value
            else:
                result[key] = val
        return result
    return {}

In [ ]:
# Step 6: OpenAI LLM call for JSON extraction from raw text
def get_llm_response(prompt):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an AI document parser. Output valid JSON only."},
            {"role": "user", "content": prompt},
        ],
        temperature=0.0,
    )
    return completion.choices[0].message.content

In [ ]:
# Step 7: Convert PDF text to JSON using LLM with given fields
def pdf_text_to_json(raw_text):
     # Step 2: Define prompt
    prompt = f"""
You are a document parser. Convert the following PDF text and form data into valid JSON.

Rules:
1. Include all fields, even if null/empty.
2. Checkboxes/radios: only include selected items. If none, set field to null.
3. Split full names into FirstName and LastName if possible.
4. Group logically related fields together.
5. Output valid JSON only, no explanations.
6. Handle data even if split across columns.

Form Fields:
{form_fields_str}

Extracted Text:
{raw_text}
"""
    return get_llm_response(prompt)

In [ ]:
# Step 8: Run everything and show combined result
if file_path:
    # Extract raw text
    raw_text = extract_text_from_pdf(file_path)
    print("----- Raw Extracted Text -----")
    print(raw_text)
    
    # Extract PDF interactive form fields
    form_fields = extract_pdf_form_fields(file_path)
    print("----- PDF Form Fields -----")
    print(form_fields)

    # Convert raw text to JSON with LLM
    extracted_json = pdf_text_to_json(raw_text)
    print("----- Extracted JSON from Text -----")
    print(extracted_json)
else:
    print("Please upload a PDF file above and rerun this cell.")